In [12]:
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
self.id = self.url.split('/')[-2]
self.raw_adress = self.html_data.find('div',{'data-name':'Geo'}).findAll('span')[0]['content']
self.metro_list = [step.text for step in self.html_data.find('div',{'data-name':'Geo'}).findAll('a')[5:]]
self.m2 = re.search('(\d{2,})',self.html_data.find('div',{'data-name':'OfferTitle'}).text).group()
self.rooms = self.html_data.find('div',{'data-name':'OfferTitle'}).text[0]
self.floor = re.search('(\d{,2})(?! из)',self.html_data.find('div',{'data-name':'ObjectSummaryDescription'}).text).group()
self.build_year = re.search('(\d{4})',self.html_data.find('div',{'data-name':'ObjectSummaryDescription'}).text).group()
self.description = self.html_data.find('p',{'itemprop':'description'}).text
self.adress_items = dict(zip(('Город','Округ','Район','Улица','Дом'),[step.text for step in self.html_data.find('div',{'data-name':'Geo'}).findAll('a')[:5]]))
self.home_items = {item.findAll('div')[0].text : item.findAll('div')[1].text for item in self.html_data.find('div',{'data-name':'BtiHouseData'}).findAll('div',{'data-name':'Item'})}
self.img_list = [img['src'] for img in self.html_data.find('div',{'data-name':'GalleryInnerComponent'}).findAll('img') if img['src'].endswith('.jpg')]


In [13]:
with open('cian.html') as b:
    html_ = BeautifulSoup(b.read())

In [ ]:
for item in html_data

In [ ]:
@asset(name = 'avito_dataframe',
       description='Выделение только новых объявлений',
       group_name='Extract')
def convert_html_2_df_cian(context,cian_page_list:list) -> pd.DataFrame:

    result_list = []
    for page in cian_page_list:
        
        items = BeautifulSoup(page, features='lxml').findAll('div',{'data-testid':'offer-card'})

        tmp_result = [get_cian_item_info(item=item) for item in items]
        result_list.extend(tmp_result)
        
    result = pd.DataFrame(result_list)

    return result


In [129]:

def get_cian_item_info(item:BeautifulSoup):
    item_desc = {}
    item_desc['datetime'] = pd.to_datetime('now',utc=True).timestamp()
    item_desc['title'] = item.find('span',{'data-mark':'OfferTitle'}).text
    item_desc['price'] = item.find('span',{'data-mark':'MainPrice'}).text.replace('\xa0₽','').replace(' ','')
    item_desc['publish_delta'] = item.find('div',{'class':'_93444fe79c--relative--IYgur'}).text.replace('\n','').replace('  ',' ')
    item_desc['url'] = item.find('div',{'data-name':'LinkArea'}).a['href']
    item_desc['id'] = item_desc['url'].split('/')[-2]
    item_desc['text'] = item.find('div',{'data-name':'Description'}).text
    item_desc.update(dict(zip(('Город','Округ','Метро','Район','Улица','Дом'),[step.text for step in item.findAll('a',{'data-name':'GeoLabel'})])))
    item_desc['metro_branch'] = item.find('div',{'data-name':'UndergroundIconWrapper'})['style'].replace('color: rgb','').replace(';','')
    item_desc['metro_name'] = [i for i in item.find('div',{'data-name':'SpecialGeo'}).text.split('\n') if i !=''][0]
    item_desc['metro_dist'] = [i for i in item.find('div',{'data-name':'SpecialGeo'}).text.split('\n') if i !=''][1]
    item_desc['img_list'] = [img['src'] for img in item.find('div',{'data-name':'Gallery'}).findAll('img') if img['src'].endswith('.jpg')]

    return item_desc

In [123]:
item_ = html_.findAll('div',{'data-testid':'offer-card'})[0]

In [ ]:
@asset(name = 'featurized_data',
       description='Дополнение данных простыми фичами',
       group_name='Featurize')
def featuring_cian_data(cian_dataframe:pd.DataFrame)->pd.DataFrame:

    cian_dataframe['price'] = cian_dataframe['price'].astype(float)
    cian_dataframe[['rooms','m2','floor']] = cian_dataframe['title'].str.split(',',expand=True)
    cian_dataframe['m2'] = cian_dataframe['m2'].str.extract(', (\d+).*м²').astype(float)
    cian_dataframe[['floor','max_floor']] = cian_dataframe['floor'].str.extract('(\d+/\d+).*эт')[0].str.split('/',expand=True).astype(int)

    cian_dataframe['text'] = cian_dataframe['text'].str.replace('\n','')
    cian_dataframe['rubm2'] = cian_dataframe['price'] / cian_dataframe['m2']
    cian_dataframe.drop(['title'],axis=1,inplace=True)

    return cian_dataframe

In [135]:
dd = pd.DataFrame([get_cian_item_info(item=item_)])

In [144]:
dd['title'].str.split(',',expand=True)[2].str.extract('(\d+/\d+).*эт')[0].str.split('/',expand=True).astype(float)

,0,1
0,12.0,15.0


In [118]:
[img['src'] for img in item_.find('div',{'data-name':'Gallery'}).findAll('img') if img['src'].endswith('.jpg')]

['https://cdn-p.cian.site/images/08/841/461/kvartira-moskovskiy-tatyanin-park-ulica-1641488041-4.jpg',
 'https://cdn-p.cian.site/images/67/841/461/kvartira-moskovskiy-tatyanin-park-ulica-1641487615-4.jpg',
 'https://cdn-p.cian.site/images/19/841/461/kvartira-moskovskiy-tatyanin-park-ulica-1641489139-4.jpg',
 'https://cdn-p.cian.site/images/99/841/461/kvartira-moskovskiy-tatyanin-park-ulica-1641489940-4.jpg',
 'https://cdn-p.cian.site/images/10/941/461/kvartira-moskovskiy-tatyanin-park-ulica-1641490155-4.jpg',
 'https://cdn-p.cian.site/images/60/941/461/kvartira-moskovskiy-tatyanin-park-ulica-1641490634-4.jpg',
 'https://cdn-p.cian.site/images/88/461/461/kvartira-moskovskiy-tatyanin-park-ulica-1641648842-4.jpg',
 'https://cdn-p.cian.site/images/12/941/461/kvartira-moskovskiy-tatyanin-park-ulica-1641492110-4.jpg',
 'https://cdn-p.cian.site/images/62/941/461/kvartira-moskovskiy-tatyanin-park-ulica-1641492636-4.jpg',
 'https://cdn-p.cian.site/images/82/941/461/kvartira-moskovskiy-tatyanin-